In [ ]:
import numpy as np
import pandas as pd
from pprint import pprint

1. Cho mảng dữ liệu sau: AA=["co","co","co","khong","khong","khong","khong"]. Viết câu lệnh cho biết có bao nhiêu loại nhãn và số lần xuất hiện.

In [ ]:
AA=np.array(["co","co","co","khong","khong","khong","khong"])
elements, count = np.unique(AA,return_counts=True)
print(elements,count)

['co' 'khong'] [3 4]


2. Viết câu lệnh tính Entropy cho mảng AA

In [ ]:
# p0 = count[0]/len(AA) #len(AA) hoặc np.sum(count)
# p1=count[1]/len(AA)
# entropy = -p0*np.log2(p0)-p1*np.log2(p1)
# print(entropy)


In [ ]:
entropy=np.sum([(-count[i]/np.sum(count))*np.log2(count[i]/np.sum(count)) for i in range(len(elements))])
entropy

0.9852281360342515

3. Viết hàm Entropy với đầu vào là một vector nhãn đầu ra là Entropy (theo công thức ở phần lý thuyết)

In [ ]:
def Entropy(y):
    elements, count = np.unique(y,return_counts=True)
    E=np.sum([(-count[i]/np.sum(count))*np.log2(count[i]/np.sum(count)) for i in range(len(elements))])
    return E

4. Viết câu lệnh đọc dữ liệu từ file weather_tv.csv. In dữ liệu để kiểm tra

In [ ]:
df = pd.read_csv('weather1.csv',encoding='latin-1')
df

,outlook,temperature,humidity,wind,class
0,sunny,hot,high,weak,no
1,sunny,hot,high,strong,no
2,overcast,hot,high,weak,yes
3,rainy,mild,high,weak,yes
4,rainy,cool,normal,weak,yes
5,rainy,cool,normal,strong,no
6,overcast,cool,normal,strong,yes
7,sunny,mild,high,weak,no
8,sunny,cool,normal,weak,yes
9,rainy,mild,normal,weak,yes


6. Tách lấy cột nhãn của dữ liệu và tính Entropy của nó.

In [ ]:
result = df.iloc[:,-1]
result

,class
0,no
1,no
2,yes
3,yes
4,yes
5,no
6,yes
7,no
8,yes
9,yes


In [ ]:
entropy=Entropy(result)
entropy

0.9402859586706311

7. Viết câu lệnh lọc lấy dữ liệu với điều kiện Thời tiết là Nắng

In [ ]:
# label = df.where(df['outlook']=='sunny').dropna()['class'] hai câu tương tự
df_sun = df[df['outlook'] == 'sunny']['class']
df_sun

,class
0,no
1,no
7,no
8,yes
10,yes


In [ ]:
en_sun=Entropy(df_sun)
en_sun

0.9709505944546686

8. Viết câu lệnh để lọc dữ liệu với độ ẩm là Cao

In [ ]:
df_hight=df[df['humidity']=='high']['class']
df_hight

,class
0,no
1,no
2,yes
3,yes
7,no
11,yes
13,no


In [ ]:
en_hight=Entropy(df_hight)
en_hight

0.9852281360342515

9. Viết câu lệnh tạo mảng chứa các loại thông tin có trong cột thời tiết (Nắng, mưa, nhiều mây) và mảng lưu số lượng

In [77]:
outlook,count_o = np.unique(df['outlook'],return_counts=True)
print(outlook,count_o)

['overcast' 'rainy' 'sunny'] [4 5 5]


In [99]:
w=0
# Cách 1:
# for i in range(len(outlook)):
#   w+=count_o[i]/np.sum(count_o)*Entropy(df.where(df['outlook']==outlook[i]).dropna()['class'])
# Cách 2:
w=np.sum([count_o[i]/np.sum(count_o)*Entropy(df.where(df['outlook']==outlook[i]).dropna()['class']) for i in range(len(outlook))])
w

0.6935361388961918

10. Viết lệnh tính Gain cho cột thời tiết.

In [103]:
g=Entropy(df.iloc[:,-1])-w
g

0.24674981977443933

11. Viết hàm tính Gain.

In [104]:
def gain(data,split_attribute_name,target_name="class"):
  total_entropy=Entropy(data[target_name])
  vals,counts= np.unique(data[split_attribute_name],return_counts=True)
  for i in range(len(vals)):
    # sum+=counts[i]/np.sum(counts)*Entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name])
    sum=np.sum([counts[i]/np.sum(counts)*Entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
  return total_entropy-sum
gain_c=gain(df,"class")
print(f"Gain(class) = {gain_c}")
gain_o=gain(df,"outlook")
print(f"Gain(outlook) = {gain_o}")

Gain(class) = 0.9402859586706311
Gain(outlook) = 0.24674981977443933


12. Viết câu lệnh lọc ra các cột có trong bảng thời tiết, trừ cột nhãn

[0.2467498197744391,
 0.029222565658954647,
 0.15183550136234136,
 0.04812703040826927]

13. Viết câu lệnh tính Gain cho từng cột trong bảng, lưu vào một mảng

14. Viết câu lệnh tình cột có Gain lớn nhất. Lưu vào cây.

In [ ]:
for i in df.columns[:-1]:
  print(f"Gain({i}) = {gain(df,i)}")

['Nhiệtđộ', 'Độẩm', 'Gió']

15. Loại cột được chọn ra khỏi mảng.